# Курсовой

- за основу брались Otvety.txt преобразованные в answers.txt
- все полезные функции вынесены в bot.py
- здесь процесс обучения и сохранения модельки / ответов / индекса
- также внизу подобие чата чтобы тестить

если запустить ```bot.py``` то можно проверить в телеграме весь конфиг вынесен в ```config.py``` но его в репозитории не будет по понятным причинам:)

мой бот в телеграме называется [gbkurs_bot](https://t.me/gbkurs_bot)

In [1]:
import annoy
import os
import pandas as pd
from gensim.models import FastText
from tqdm import notebook

from bot import ROOT, EMBEDING_COUNT, getVector, preprocess_txt, clean_str, getResponse
from bot import MINDESTINATION, MAXDESTINATION

In [2]:
if not os.path.exists(f'{ROOT}df.pkl'):
    tmp = []
    with open("../data/answers.txt", "r") as f:
        for line in notebook.tqdm(f):
            tb = line.split("\t")
            q = preprocess_txt(tb[0])
            if len(q) > 2:
                a = clean_str(tb[1])
                tmp.append([q, a])
    df = pd.DataFrame(tmp, columns=['question', 'answer'])
    df.to_pickle(f'{ROOT}df.pkl')
else:
    df = pd.read_pickle(f'{ROOT}df.pkl')

In [3]:
if not os.path.exists(f'{ROOT}ft_model'):
    model = FastText(sentences=df.question.values, size=EMBEDING_COUNT, min_count=10, window=3)
    model.save(f'{ROOT}ft_model')
else:
    model = FastText.load(f'{ROOT}ft_model')

In [4]:
index = annoy.AnnoyIndex(EMBEDING_COUNT ,'angular')
if not os.path.exists(f'{ROOT}speaker.ann'):
    for i, question in enumerate(df.question.values):
        index.add_item(i, getVector(question, model))
    index.build(10)
    index.save(f'{ROOT}speaker.ann')
else:
    index.load(f'{ROOT}speaker.ann')

In [5]:
q = 'start'
while q != '':
    q = input()
    text, dst = getResponse(q, model, index, df.answer)
    if dst <= MAXDESTINATION:
        print(f'{dst:0.3f}: {text}')

привет
0.309: Привет как ты  
я нормально
0.496: Такое бывает 
расскажи мне что-нибудь
0.304: Возвращается женщина после работы домой  уставшая  сумки с продуктами тяжеленные  заходит в подъезд  ... хотя все это ерунда
интересно а что еще ты можешь рассказать?
0.273: Представляешь  никто даже не попытался      разве что здесь  на сайте ответов    звание  высшего ... хотя все это ерунда
что не пытался ?
0.526:    а он вообще что понимает     ему тем более всё  по барабану    
кому ему 
0.371: Я завидую тольок себе самой  . Хоть бы денёк никому не нужна была    мечтаю    добрый вечерок    
модель обучена ужасно
0.566: Ты пришёл на работу  а тебе за вчерашнее вы уволены  . Www livemocha com посмотри 
ну вот еще и уволила
0.410: Применить дисциплинарное взыскание в виде увольнения  при этом не забывая соблюсти процедуру ст  193 ... хотя все это ерунда
хоть где то попадает примерно
0.484: Ноги окривеют таскать такой вес . Либо винтовка дрянь либо пульки разновес  
гыгы
гы гы гы
ну и ладно
0